In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn

from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image

In [ ]:
latent_dim = 100


# 생성자(Generator) 클래스 정의
class Generator(nn.Module):
    
    def __init__(self):
        super(Generator, self).__init__()

        # 하나의 블록(block) 정의
        def block(input_dim, output_dim, normalize=True):
            layers = [nn.Linear(input_dim, output_dim)]
            if normalize:
                # 배치 정규화(batch normalization) 수행(차원 동일)
                layers.append(nn.BatchNorm1d(output_dim, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        # 생성자 모델은 연속적인 여러 개의 블록을 가짐
        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, 1 * 28 * 28),
            nn.Tanh()
        )

    def forward(self, z): # 노이즈 벡터 z
        img = self.model(z)
        # 이미지 형태로 만들어줌
        img = img.view(img.size(0), 1, 28, 28) # 배치사이즈, 채널, 넓이, 높이
        return img

In [ ]:
from torch.nn.modules import flatten
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator,self).__init__()
    self.model = nn.Sequential(
        nn.Linear(1*28*28,512),
        nn.LeakyReLU(0.2,inplace=True),
        nn.Linear(512,256),
        nn.LeakyReLU(0.2,inplace=True),
        nn.Linear(256,1),
        nn.Sigmoid(),
    )
  def forward(self,img):
    flattened = img.view(img.size(0),-1)
    output = self.model(flattened)

    return output

In [ ]:
transforms_train = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

train_dataset = datasets.MNIST(root = './dataset', train=True, download=True, transform=transforms_train)
dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=128, shuffle=True, num_workers=4)

In [ ]:
generator = Generator()
discriminator = Discriminator()

generator.cuda()
discriminator.cuda()

adversarial_loss = nn.BCELoss()
adversarial_loss.cuda()

lr = 0.0002

optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5,0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5,0.999))


In [ ]:
import time

n_epochs = 200 # 학습의 횟수(epoch) 설정
sample_interval = 2000 # 몇 번의 배치(batch)마다 결과를 출력할 것인지 설정
start_time = time.time()

for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # 진짜(real) 이미지와 가짜(fake) 이미지에 대한 정답 레이블 생성
        real = torch.cuda.FloatTensor(imgs.size(0), 1).fill_(1.0) # 진짜(real): 1
        fake = torch.cuda.FloatTensor(imgs.size(0), 1).fill_(0.0) # 가짜(fake): 0

        real_imgs = imgs.cuda()

        """ 생성자(generator)를 학습합니다. """
        optimizer_G.zero_grad()

        # 랜덤 노이즈(noise) 샘플링
        z = torch.normal(mean=0, std=1, size=(imgs.shape[0], latent_dim)).cuda()

        # 이미지 생성
        generated_imgs = generator(z)

        # 생성자(generator)의 손실(loss) 값 계산
        g_loss = adversarial_loss(discriminator(generated_imgs), real)

        # 생성자(generator) 업데이트
        g_loss.backward()
        optimizer_G.step()

        """ 판별자(discriminator)를 학습합니다. """
        optimizer_D.zero_grad()

        # 판별자(discriminator)의 손실(loss) 값 계산
        real_loss = adversarial_loss(discriminator(real_imgs), real)
        fake_loss = adversarial_loss(discriminator(generated_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        # 판별자(discriminator) 업데이트
        d_loss.backward()
        optimizer_D.step()

        done = epoch * len(dataloader) + i
        if done % sample_interval == 0:
            # 생성된 이미지 중에서 25개만 선택하여 5 X 5 격자 이미지에 출력
            save_image(generated_imgs.data[:25], f"{done}.png", nrow=5, normalize=True)

    # 하나의 epoch이 끝날 때마다 로그(log) 출력
    print(f"[Epoch {epoch}/{n_epochs}] [D loss: {d_loss.item():.6f}] [G loss: {g_loss.item():.6f}] [Elapsed time: {time.time() - start_time:.2f}s]")

[Epoch 0/200] [D loss: 0.432960] [G loss: 1.424452] [Elapsed time: 7.88s]
[Epoch 1/200] [D loss: 0.501635] [G loss: 1.950237] [Elapsed time: 13.32s]
[Epoch 2/200] [D loss: 0.303494] [G loss: 1.074245] [Elapsed time: 18.95s]
[Epoch 3/200] [D loss: 0.355045] [G loss: 2.027945] [Elapsed time: 24.37s]
[Epoch 4/200] [D loss: 0.398817] [G loss: 2.944833] [Elapsed time: 29.90s]
[Epoch 5/200] [D loss: 0.301190] [G loss: 2.080935] [Elapsed time: 35.57s]
[Epoch 6/200] [D loss: 0.448281] [G loss: 0.745987] [Elapsed time: 41.17s]
[Epoch 7/200] [D loss: 0.209357] [G loss: 1.440529] [Elapsed time: 46.84s]
[Epoch 8/200] [D loss: 0.309366] [G loss: 1.123897] [Elapsed time: 52.34s]
[Epoch 9/200] [D loss: 0.318698] [G loss: 1.777354] [Elapsed time: 57.72s]
[Epoch 10/200] [D loss: 0.241701] [G loss: 2.034793] [Elapsed time: 63.24s]
[Epoch 11/200] [D loss: 0.401532] [G loss: 0.868858] [Elapsed time: 68.70s]
[Epoch 12/200] [D loss: 0.292676] [G loss: 1.671215] [Elapsed time: 74.33s]
[Epoch 13/200] [D loss:

In [ ]:
from IPython.display import Image

Image('92000.png')